In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = 'true'

os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com'

os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

In [3]:
import bs4
from langchain  import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

#### INDEXING ####

# Load Documents

loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vector_store = Chroma.from_documents(documents=splits, embedding=GoogleGenerativeAIEmbeddings(model= "models/embedding-001", google_api_key=os.getenv("GEMINI_API_KEY")))

retriever = vector_store.as_retriever()

# RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join([f"{doc.page_content}" for doc in docs])

# Chain
rag_chain = (
    {"context": retriever | format_docs,
     "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("what is Task Decomposition?")

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\FIDELIS ALIU\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks. This approach, often used in conjunction with Chain of Thought prompting, helps AI models understand and execute complex tasks by addressing them step-by-step.  This can be achieved through simple prompting techniques, task-specific instructions, or human input. \n'

# Part 2: Indexing

In [4]:
# Documents
question = "What kinds of pets do I like?"
document = "My favourite pet is a cat."

In [5]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

Text embedding models

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embd = GoogleGenerativeAIEmbeddings(model= "models/embedding-001", google_api_key=os.getenv("GEMINI_API_KEY"))
query_result = embd.embed_query(question)
documnt_result = embd.embed_query(document)
len(query_result), len(documnt_result)

(768, 768)

cosine similarity (1 indicates identical)

In [7]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, documnt_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.8456968223547316


Document Loaders

In [8]:
### INDEXING ####

# load blog
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

splitter

> This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chuks are small enough. The default list is ["\n\n", "\n", " ", ""]. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest related semantically related pieces of text.

In [9]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 300, 
    chunk_overlap=50
)

# Make splits
splits = text_splitter.split_documents(blog_docs)

Vectorstores

In [10]:
# Index
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

vector_store = Chroma.from_documents(
    documents=splits, 
    embedding=GoogleGenerativeAIEmbeddings(model= "models/embedding-001", google_api_key=os.getenv("GEMINI_API_KEY"))
)

retriever = vector_store.as_retriever()

# Part 3: Retrieval

In [11]:
# Index
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
vector_store = Chroma.from_documents(
    documents=splits, 
    embedding=GoogleGenerativeAIEmbeddings(model= "models/embedding-001", google_api_key=os.getenv("GEMINI_API_KEY"))
)

retriever = vector_store.as_retriever(search_kwargs={"k": 1})

# k is the number of nearby neighbors you want to return

In [12]:
docs = retriever.get_relevant_documents("What is Task Decomposition?")

C:\Users\FIDELIS ALIU\AppData\Local\Temp\ipykernel_4512\4059233835.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  docs = retriever.get_relevant_documents("What is Task Decomposition?")


In [13]:
len(docs)

1

# Part 4: Generation

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

# Prompt 
template = """Answer the question based only on the following context
{context}

Question: {question}
"""

prompt =  ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context\n{context}\n\nQuestion: {question}\n'))])

In [15]:
# LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0)

In [16]:
# Chain
chain = prompt | llm

In [17]:
# Run
chain.invoke({"context": docs, "question": "What is Task Decomposition?"})

AIMessage(content='Task decomposition is a technique used by LLM-powered autonomous agents to break down complex tasks into smaller, more manageable steps. This can be achieved through various methods:\n\n1. **Chain of thought (CoT) prompting:** Instructing the model to "think step by step," allowing it to decompose the task into simpler subtasks.\n2. **Tree of Thoughts (ToT):** Expanding on CoT by exploring multiple reasoning possibilities at each step, creating a tree-like structure of potential solutions.\n3. **LLM prompting:** Using prompts like "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?" to guide the model in breaking down the task.\n4. **Task-specific instructions:** Providing instructions tailored to the specific task, such as "Write a story outline" for novel writing.\n5. **Human input:** Directly providing the task decomposition steps to the model. \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'ST

In [18]:
from langchain import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [19]:
prompt_hub_rag

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

RAG chains

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever,
     "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("what is Task Decomposition?")

'Task decomposition is a technique used by LLM-powered autonomous agents to break down complex tasks into smaller, more manageable steps. This is often achieved through prompting techniques like Chain of Thought (CoT) where the model is instructed to "think step by step". \n\nHere\'s how task decomposition can be done:\n\n1. **Simple Prompting:** Asking the LLM to list steps or subgoals for a task (e.g., "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?")\n2. **Task-Specific Instructions:** Providing instructions tailored to the task, like "Write a story outline" for novel writing.\n3. **Human Input:** Directly involving humans in breaking down the task. \n'

# Part 5: Multi Query

In [21]:
#### INDEXXING ####


# load blog
import bs4
loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("post-content", "post-title", "post-header")
        )
    ),
)

blog_docsdocs = loader.load()


# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 300, 
    chunk_overlap=50
)

# Make splits
splits = text_splitter.split_documents(blog_docs)


# Index
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
vector_store = Chroma.from_documents(
    documents=splits, 
    embedding=GoogleGenerativeAIEmbeddings(model= "models/embedding-001", google_api_key=os.getenv("GEMINI_API_KEY"))
)

retriever = vector_store.as_retriever()


### Prompt

In [22]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help 
the user overcome some of the limitaions of the distance-based similarity search. 
Provide the alternative questions separated by newlines. Original Question: {question}"""

prompt_prespectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

generate_queries = (
    prompt_prespectives
    | ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [23]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})
len(docs)

6

In [24]:
from operator import itemgetter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough

# RAG

template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0)

final_rag_chain = (
    {
        "context": retrieval_chain,
        "question": itemgetter("question")
    }
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'Task decomposition is a crucial process for LLM-powered agents to handle complex tasks. It involves breaking down a large, intricate task into a sequence of smaller, more manageable subtasks. This approach makes it easier for the LLM to understand and address the problem systematically. \n\nHere\'s a breakdown of task decomposition in LLM agents:\n\n**Methods of Task Decomposition:**\n\n* **Chain-of-Thought (CoT) Prompting:** This technique encourages the LLM to "think step-by-step" by explicitly prompting it to outline the steps required to complete the task. This helps in decomposing the problem into smaller, more interpretable steps.\n* **Tree of Thoughts (ToT):**  An extension of CoT, ToT explores multiple reasoning possibilities at each step, creating a tree-like structure of potential solutions. This allows for a broader exploration of the solution space and can lead to more robust solutions.\n* **LLM-based Decomposition:** LLMs can be directly prompted to decompose tasks using 

# Part 6: RAG-Fusion

In [25]:
from langchain.prompts import ChatPromptTemplate

# RAF-Fusion; Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

generate_queries = (
    prompt_rag_fusion
    | ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [27]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal Rank Fusion that takes multiple lists of ranked documents
        and an optional parameter k used in the RRF formula. """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate over each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its  rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the documet to a string format to use as  a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with a score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any 
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1/(rank * k)
            fused_scores[doc_str] += 1 / (rank + k)
        
    # Sort the documents by their fused scores in descending order to get the final ranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

5

In [28]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {
        "context": retrieval_chain_rag_fusion,
        "question": RunnablePassthrough() # I'm testing to see if it'll work instead of itemgetter("question")
    }
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'Task decomposition for LLM agents is the process of breaking down a complex task into smaller, more manageable subtasks. This allows the agent to tackle the problem more efficiently and effectively. \n\nHere are some key methods of task decomposition mentioned in the context:\n\n* **Chain of Thought (CoT):** This technique prompts the LLM to "think step by step," decomposing the task into a sequence of smaller steps.\n* **Tree of Thoughts:** This method expands on CoT by exploring multiple reasoning possibilities at each step, creating a tree-like structure of potential solutions.\n* **Prompt Engineering:**  Simple prompts like "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?" can guide the LLM to decompose the task.\n* **Task-Specific Instructions:**  Providing instructions tailored to the specific task, such as "Write a story outline" for writing a novel, can aid in decomposition.\n* **Human Input:**  Humans can directly assist in breaking down the task into subtas

# Part 7: Decomposition

In [29]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answered in isolation. \n
Generate multiple search queries related to: {question} \n
Only return the sub-questions \n
Output (3 queries):"""

prompt_decomposition = ChatPromptTemplate.from_template(template)

In [30]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run 
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question": question})

In [31]:
questions

['1. What are the core functions of an autonomous agent? ',
 '2. How do Large Language Models (LLMs) contribute to autonomous agent capabilities?',
 '3. What are the typical architectural components found in an LLM-powered autonomous agent system? ',
 '']

In [32]:
# Prompt 
template = """Here is the question you need to answer:
\n --- \n {question} \n --- \n

Here is any available background question - answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question:

\n --- \n {context} \n --- \n

Use the above context and any background question - answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [33]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """ Format Q and A pairs """

    formatted_string = ""
    formatted_string += f"Question: {question}  \nAnswer: {answer} \n\n"
    return formatted_string.strip()

# LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0)

#
q_a_pairs =""
for q in questions:
    rag_chain = (
        {
            "context": itemgetter("question") | retriever,
            "question": itemgetter("question"),
            "q_a_pairs": itemgetter("q_a_pairs")
        }
        | decomposition_prompt
        | llm
        | StrOutputParser()
    )

    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})
    q_a_pair = format_qa_pair(q, answer)
    q_a_pairs = q_a_pairs + "\n --- \n" + q_a_pair

In [34]:
answer

'The provided text snippets offer a glimpse into the capabilities of autonomous agents, particularly those powered by Large Language Models (LLMs), but they don\'t directly answer a specific question. \n\nHowever, we can infer that the snippets relate to the concept of **goal-oriented autonomous agents** and their potential applications. \n\nHere\'s a breakdown:\n\n1. **Goal-Driven Behavior:** The repeated mention of "GOALS" followed by placeholders for "user-provided goal" strongly suggests a system designed to pursue and achieve specific objectives defined by the user.\n\n2. **LLM as the Core:** While not explicitly stated, the agent\'s ability to "inquire," "select," "request," and "attempt synthesis" implies a level of language understanding, reasoning, and decision-making that aligns with the capabilities of LLMs.\n\n3. **Scientific Discovery Example:** The example of an agent working in drug discovery ("inquired about current trends in anticancer drug discovery," "selected a targ

Answer individually

In [35]:
# AAnswer each sub-question individually

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):
    """ RAG on each sub-question """

    # Use our decomposition
    sub_questions = sub_question_generator_chain.invoke({"question": question})

    # Initialize a list to hold RAG chain results
    rag_results = []

    for sub_question in sub_questions:

        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)

        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({
            "context": retrieved_docs,
            "question": sub_question
        })

        rag_results.append(answer)

    return rag_results, sub_questions

# wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [36]:
def format_qa_pairs(questions, answers):
    """ Format Q and A pairs """

    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}  \nAnswer {i}: {answer} \n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = ( prompt | llm | StrOutputParser() )

final_rag_chain.invoke({"context": context, "question": question})

"An LLM-powered autonomous agent system can be broken down into several key components:\n\n**1. The Large Language Model (LLM):** This acts as the **brain** of the system. It provides the intelligence to understand instructions, break down complex tasks into smaller sub-goals, and generate plans to achieve those goals. The LLM's ability to learn from its actions and improve over time is crucial for the agent's autonomy.\n\n**2. Goal Management System:** This component defines the agent's objectives and constraints. It could be a simple set of instructions or a more complex system that dynamically adjusts goals based on the environment and feedback. \n\n**3. Action Execution Engine:** This component translates the LLM's plans into concrete actions within a specific environment. For example, it might interact with APIs, databases, or even physical systems depending on the agent's purpose.\n\n**4. Feedback and Learning Mechanism:** This component allows the agent to learn from its experie

# Part 8: Step Back

In [37]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?"
    },
    {
        "input": "Jan Sindel's was born in what country?",
        "output": "what is Jan Sindel's personal history?"
    },
]

# We now transform these to example messages

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:"""
        ),
        # Few-shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}")
    ]
)

In [38]:
generate_queries_step_back = prompt | ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'What is task decomposition? \n'

In [39]:
# Response prompt
response_prompt_template = """You are an expert at world knowledge. I am going to ask you a question. Your response should be conprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer: """

response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"]
    }
    | response_prompt
    | ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0)
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition is a crucial component in building LLM-powered autonomous agents, enabling them to tackle complex tasks effectively. It involves breaking down a large, intricate task into a sequence of smaller, more manageable subtasks. This approach offers several benefits:\n\n* **Improved Performance:** LLMs often struggle with complex instructions. Decomposing tasks into smaller steps allows the model to focus on one aspect at a time, leading to better overall performance.\n* **Enhanced Reasoning:** By explicitly outlining the steps involved, task decomposition encourages the LLM to engage in a more structured and logical reasoning process, similar to how humans solve problems.\n* **Interpretability:**  Breaking down the task makes the LLM\'s thought process more transparent, allowing developers to understand how the model arrived at a particular solution and identify potential areas for improvement.\n\nHere are some key methods for task decomposition in LLM agents:\n\n**1. Chai

# Part 9: HyDE

In [40]:
from langchain.prompts import ChatPromptTemplate

# HyDE document operation
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""

prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0)

generate_docs_for_retrieval = prompt_hyde | model | StrOutputParser()

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question": question})

"## Task Decomposition for Large Language Model Agents: A Bridge to Complex Action\n\nLarge language models (LLMs) have demonstrated remarkable abilities in understanding and generating human-like text. However, their application as agents capable of interacting with complex environments and achieving multi-step goals requires a mechanism to translate high-level instructions into actionable steps. This mechanism is known as **task decomposition**. \n\nTask decomposition for LLM agents involves breaking down a complex task, often expressed in natural language, into a sequence of smaller, more manageable subtasks. These subtasks are designed to be atomic, meaning they can be executed by the agent directly or through simple API calls. This decomposition process effectively bridges the gap between the LLM's high-level understanding of the task and the granular actions required for its execution.\n\nSeveral approaches to task decomposition exist, ranging from predefined hierarchical structu

In [41]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever
retrieved_docs = retrieval_chain.invoke({"question": question})
retrieved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search

In [42]:
# RAG

template = """Answer the following question based on this context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = prompt | model | StrOutputParser()

final_rag_chain.invoke({"context": retrieved_docs, "question": question})

'Task decomposition for LLM agents is the process of breaking down a complex task into smaller, more manageable subtasks. This allows the agent to tackle the problem more efficiently and effectively. \n\nHere\'s how it works:\n\n* **Chain of Thought (CoT):** The LLM is prompted to "think step-by-step," decomposing the task into a sequence of smaller steps. This helps the model understand and solve complex problems by breaking them down.\n* **Tree of Thoughts:** This method expands on CoT by exploring multiple reasoning possibilities at each step. It generates a tree-like structure of potential solutions, allowing for a more comprehensive exploration of the problem space.\n* **Methods of Decomposition:** Task decomposition can be achieved through:\n    * **Simple Prompting:** Using prompts like "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?"\n    * **Task-Specific Instructions:** Providing instructions tailored to the specific task, such as "Write a story outline" fo